- mudar req.minuto_a_minuto: descrição --> descricao
- Banco de dados para jogadores: Novo requerimento no site ESPN
- No link resultados (https://www.espn.com.br/futebol/resultados/_/liga/eng.1) de um campeonato qualquer, os dias em que ocorreram jogos são marcados em negrito. Verificar se podemos tirar proveito desse fato.
- Garantir a igualdade dos nomes dos jogadores, definidos em req.minuto_a_minuto e req.jogadores
- Analisar separação: descrição assistência - descrição gol
- Verificar métodos que geram as descrições, existem casos em que a função retorna None.

In [33]:
from pyspark.sql import SparkSession

In [34]:
spark = SparkSession.builder.appName('Futebol').getOrCreate()

In [4]:
import pandas as pd
import numpy as np

In [12]:
from RequisiçãoESPN import RequisiçãoFutbeol

import re

In [70]:
padrao = [r'gol!.+[0-9]+,.+[0-9]+\.', r'gol contra marcado por']

pos = []
for i in range(len(req.minuto_a_minuto)):
    exp = req.minuto_a_minuto[i]['descrição']
    if (re.search(padrao[0],exp, flags=re.I) != None) or (re.search(padrao[1],exp, flags=re.I) != None):
        print(i,req.minuto_a_minuto[i]['tempo'])
        pos.append(i)

9 85
38 60
53 0
65 35
85 20


In [13]:
exemplos_gols = [{'jogo_id': '600630', 'posicao': [30,37,81]},
                 {'jogo_id': '581874', 'posicao': [23,35,38,43,44]},
                 {'jogo_id': '581882', 'posicao': [20, 23, 37, 78, 87, 89]},
                 {'jogo_id': '581881', 'posicao': [49, 75, 88, 90]},
                 {'jogo_id': '578332', 'posicao': [9, 51, 73, 77]},
                 {'jogo_id': '578340', 'posicao': [4, 56, 85, 103]},
                 {'jogo_id': '578342', 'posicao': [2, 15, 49, 51]},
                 {'jogo_id': '578350', 'posicao': [15, 18, 22, 55, 60]},
                 {'jogo_id': '578348', 'posicao': [26, 31, 63, 66]},
                 {'jogo_id': '582824', 'posicao': [24, 25, 48, 56, 60, 74, 87]},
                 {'jogo_id': '572939', 'posicao': [76, 77, 114, 115]},
                 {'jogo_id': '572934', 'posicao': [20, 27, 66, 77, 80]},
                 {'jogo_id': '572936', 'posicao': [4, 17, 29, 32, 91, 97]},
                 {'jogo_id': '572937', 'posicao': [9, 38, 53, 65, 85]}
                ]

In [4]:
padrao = [r'gol!.+[0-9]+,.+[0-9]+\.', r'gol contra marcado por']

for ex_gol in exemplos_gols:
    
    jogo_id =  ex_gol['jogo_id']
    req = RequisiçãoFutbeol(jogo_id)
    req.req_minuto_a_minuto()
    
    for pos in ex_gol['posicao']:
        exp = req.minuto_a_minuto[pos]['descrição']
        if (re.search(padrao[1],exp, flags=re.I) != None):
            print(exp)
            print(jogo_id, pos)
            print()

TypeError: 'NoneType' object is not subscriptable

In [14]:
# Padrões: Gol e finalização

padrao_finalizacao = [r'pé esquerdo', r'pé direito', r'de cabeça']
padrao_local_finalizacao = [r'meio da área', r'fora da área', r'lado esquerdo da área', 
                            r'lado direito da área', r'muito perto', r'lado direito da pequena área',
                           r'lado esquerdo da pequena área', r'ângulo dificil da direita', r'ângulo dificil da esquerda']
padrao_lado_gol = [r'lado direito do gol', r'lado esquerdo do gol', r'ângulo superior esquerdo',
                   r'ângulo superior direito', r'centro do gol', r'canto inferior esquerdo',
                   r'canto inferior direito', r'lado inferior direito do gol', r'lado inferior esquerdo do gol',
                   r'centro superior do gol', r'centro inferior do gol', r'oportunidade perdida', r'finalização bloqueada']
padrao_desc_assistencia = [r'cruzamento', r'passe em profundidade', r'passe de cabeça']
padrao_origem_finalizacao = [r'após escanteio', r'bola parada', r'contra-ataque']
padrao_inicio_desc = [r'com ', r'depois ', r'de bola', r'após']

class Gol():
    
    def __init__(self, evento, jogo_id, index):
        
        self.jogo_id = None
        self.index = None
        self.time_casa = None
        self.time_visitante = None
        self.tempo = None
        self.bin_penalti = None
        self.bin_falta = None
        self.bin_gol_contra = None
        self.time = None
        self.placar = None
        self.autor_gol = None
        self.finalizacao = None
        self.local_finalizacao = None
        self.lado_gol = None
        self.autor_assistencia = None
        self.descricao_assistencia = None
        self.origem_finalizacao = None
        
        exp = evento['descrição']
        padrao = [r'gol!.+[0-9]+,.+[0-9]+\.', r'gol contra marcado por']
        if (re.search(padrao[0],exp, flags=re.I) != None) or (re.search(padrao[1],exp, flags=re.I) != None):
            self.bin_gol = True
        else:
            self.bin_gol = False
        
        if self.bin_gol:
            
            # jogo_id
            self.jogo_id = jogo_id
            
            # index
            self.index = index
            
            # time da casa e time visitante
            padrao = r'[!\.].+[0-9]+,.+[0-9]+\.'
            exp_1 = re.findall(padrao,exp,flags=re.I)[0][2:]
            padrao_1 = r'.+[0-9],'
            padrao_2 = r',.+[0-9]'
            self.time_casa = re.findall(padrao_1,exp_1,flags=re.I)[0][:-3].replace('! ','')
            self.time_visitante = re.findall(padrao_2,exp_1,flags=re.I)[0][2:-2]
            
            # tempo
            self.tempo = evento['tempo']
            
            # penalti - variável binária
            self.bin_penalti = (re.search(r'p[eê]nalti',exp, flags=re.I) != None)
            
            # falta - variável binária
            self.bin_falta = (re.search(r'gol de falta',exp, flags=re.I) != None)
            
            # gol_contra - variável binária
            self.bin_gol_contra = (re.search(r'gol contra marcado por',exp, flags=re.I) != None)
            
            # time e placar
            if self.bin_gol_contra:
                qt_time_casa = len(re.findall(self.time_casa,exp,flags=re.I))
                qt_time_visitante = len(re.findall(self.time_visitante,exp,flags=re.I))
                if qt_time_casa == 2 and qt_time_visitante == 1:
                    self.time = self.time_visitante
                elif qt_time_casa == 1 and qt_time_visitante == 2:
                    self.time = self.time_casa
                else:
                    return 'Erro no procedimento: time.'
                padrao = r'[!\.].+[0-9]+,.+[0-9]+\.'
                exp_1 = re.findall(padrao,exp,flags=re.I)[0][2:]
            else:
                padrao = r'gol!.+[0-9]+,.+[0-9]+\.'
                exp_1 = re.findall(padrao,exp,flags=re.I)[0]
                exp_2 = exp.replace(exp_1+' ','')
                self.time = re.findall(r'\(.+\)',exp_2,flags=re.I)[0][1:-1]
            placar = re.findall('[0-9]+',exp_1,flags=re.I)
            self.placar = [int(p) for p in placar]
            
            # autor_gol
            if self.bin_gol_contra:
                padrao = r'Gol contra marcado por'
                padrao_1 = padrao + '.+' + self.time_casa
                exp_1 = re.findall(padrao_1,exp,flags=re.I)[0]
                padrao_2 = padrao + '.+,'
                self.autor_gol = re.findall(padrao_2,exp_1,flags=re.I)[0].replace(padrao,'')[1:-1]
            else:
                padrao = r'gol!.+?[0-9]+,.+?[0-9]+\.'
                exp_1 = re.findall(padrao,exp,flags=re.I)[0]
                exp_2 = exp.replace(exp_1+' ','')
                self.autor_gol = re.findall(r'.+?\(', exp_2,flags=re.I)[0][:-2]
                
            # finalizacao, local_finalizacao e lado_gol
            try:
                padrao = r'gol!.+?[0-9]+,.+?[0-9]+\.'
                exp_1 = re.findall(padrao,exp,flags=re.I)[0]
                exp_2 = exp.replace(exp_1+' ','')
            except:
                pass
            for p in padrao_finalizacao:
                try:
                    self.finalizacao = re.findall(p,exp_2,flags=re.I)[0]
                except:
                    pass
            for p in padrao_local_finalizacao:
                try:
                    self.local_finalizacao = re.findall(p,exp_2,flags=re.I)[0]
                except:
                    pass
            for p in padrao_lado_gol:
                try:
                    self.lado_gol = re.findall(p,exp_2,flags=re.I)[0]
                except:
                    pass
            
            # autor_assistencia
            exp_1 = exp.replace(re.findall(r'.+?\..+?\.',exp,flags=re.I)[0], '')
            self.autor_assistencia = re.sub(r' *assistência de ', '',exp_1, flags = re.I).replace('.','')
            if self.autor_assistencia == '':
                self.autor_assistencia = None
            for p in padrao_inicio_desc:
                 if re.search(p, exp_1, flags=re.I) != None:
                    self.autor_assistencia = re.sub(p+'.+','', self.autor_assistencia,flags=re.I)[:-1]  
                    break
                    
            # descricao_assistencia
            exp_1 = exp.replace(re.findall(r'.+?\..+?\.',exp,flags=re.I)[0], '')
            for p in padrao_desc_assistencia:
                try:
                    self.descricao_assistencia = re.findall(p, exp_1,flags=re.I)[0]
                except:
                    pass
                    
            # origem_gol
            for p in padrao_origem_finalizacao:
                try:
                    self.origem_finalizacao = re.findall(p, exp,flags=re.I)[0]
                except:
                    pass
                
    
    def gerar_df(self):
        
        if self.bin_gol:
        
            columns = ['jogo_id', 
                       'index',
                       'time_casa',
                       'time_visitante',
                       'tempo', 
                       'penalti', 
                       'falta', 
                       'gol_contra',
                       'time', 
                       'placar',
                       'autor_gol', 
                       'finalizacao', 
                       'local_finalizacao', 
                       'lado_gol',
                       'autor_assistencia',
                       'descricao_assistencia',
                       'origem_finalizacao']
            
            data = [self.jogo_id, 
                    self.index,
                    self.time_casa, 
                    self.time_visitante, 
                    self.tempo,
                    self.bin_penalti,
                    self.bin_falta,
                    self.bin_gol_contra,
                    self.time,
                    self.placar,
                    self.autor_gol,
                    self.finalizacao,
                    self.local_finalizacao,
                    self.lado_gol, 
                    self.autor_assistencia,
                    self.descricao_assistencia,
                    self.origem_finalizacao]
            
            self.df = pd.DataFrame(data = data).transpose()
            self.df.columns = columns
        
        else:
            return 'Erro! O evento inserido não identifica um gol.'

In [ ]:
columns = ['jogo_id', 
           'time_casa',
           'time_visitante',
           'tempo', 
           'penalti', 
           'falta', 
           'gol_contra',
           'time', 
           'placar',
           'autor_gol', 
           'finalizacao', 
           'local_finalizacao', 
           'lado_gol',
           'autor_assistencia',
           'descricao_assistencia',
           'origem_gol',
           'i',
           'j']

df_gols = pd.DataFrame(columns = columns)

for i in range(len(exemplos_gols)):
   
    g = exemplos_gols[i]
    jogo_id =  g['jogo_id']
    req = RequisiçãoFutbeol(jogo_id)
    while req.minuto_a_minuto == None or req.gols == None:
        req.req_minuto_a_minuto()
        req.req_gols()
    
    for j in range(len(g['posicao'])):
        
        pos = g['posicao'][j]
        evento = req.minuto_a_minuto[pos]

        gol = Gol(evento, jogo_id, 10)
        gol.gerar_df()
        df = gol.df
        df['i'] = i
        df['j'] = j
        df_gols = pd.concat([df_gols, df])
        
df_gols.index = range(df_gols.shape[0])

In [ ]:
# Verificado até linha 50

# linnha 34: descrição gol (após escanteio) não incluida na tabela
# linha 47: descrição gol (depois de um contra-ataque) não incluida na tabela
# linha 51: descrição gol (de bola parada) não incluida na tabela

for l in range(df_gols.shape[0]):
    
    i,j = df_gols['i'][l], df_gols['j'][l]
    if l > 0:
        i_0 = df_gols['i'][l-1]
    else:
        i_0 = -1
        
    g = exemplos_gols[i]
    jogo_id =  g['jogo_id']
    pos = g['posicao'][j]
    
    if i > i_0:
        req = RequisiçãoFutbeol(jogo_id)
        req.req_minuto_a_minuto()
    
    print(l+1)
    print(req.minuto_a_minuto[pos])
    for c in df_gols.columns:
        if df_gols[c][l] != None and c != 'i' and c != 'j':
            print(c + ':',df_gols[c][l])
    print('--------------------------------------------')

In [15]:
padrao_inicio_autor_finalizacao = [r'\.',r'!',r',', r'oportunidade perdida']

class Finalizacao():
    
    def __init__(self, evento, jogo_id, index):
        
        self.jogo_id = None
        self.index = None
        self.tempo = None
        self.bin_penalti = None
        self.time = None
        self.autor_finalizacao = None        
        self.finalizacao = None
        self.local_finalizacao = None
        self.lado_gol = None
        self.autor_assistencia = None
        self.descricao_assistencia = None
        self.origem_finalizacao = None
        
        exp = evento['descrição']
        bin_gol = Gol(evento, jogo_id, index).bin_gol
        
        padrao = [r'finalização', r'oportunidade perdida']
        if bin_gol:
            self.bin_finalizacao = False
        elif (re.search(padrao[0],exp, flags=re.I) != None) or (re.search(padrao[1],exp, flags=re.I) != None):
            self.bin_finalizacao = True
        else:
            self.bin_finalizacao = False
            
        if self.bin_finalizacao:
            
            # jogo_id
            self.jogo_id = jogo_id
            
            # index
            self.index = index
            
            # tempo
            self.tempo = evento['tempo']
            
            # penalti - variável binária
            self.bin_penalti = (re.search(r'p[eê]nalti',exp, flags=re.I) != None)
            
            # time 
            self.time = re.findall(r'\(.+?\)',exp,flags=re.I)[0][1:-1]
            
            # autor_finalizacao
            for p in padrao_inicio_autor_finalizacao:
                try:
                    self.autor_finalizacao = re.findall(p+'.+?\(',exp,flags=re.I)[0][:-2]
                    self.autor_finalizacao = re.sub(p,'', self.autor_finalizacao,flags=re.I)[1:]  
                except:
                    pass
            
            # finalizacao, local_finalizacao e lado_gol
            exp_2 = re.sub(r'assistência .+','', exp, flags=re.I)
            for p in padrao_finalizacao:
                try:
                    self.finalizacao = re.findall(p, exp_2, flags=re.I)[0]
                except:
                    pass
            for p in padrao_local_finalizacao:
                try:
                    self.local_finalizacao = re.findall(p, exp_2, flags=re.I)[0]
                except:
                    pass
            for p in padrao_lado_gol:
                try:
                    self.lado_gol = re.findall(p, exp_2, flags=re.I)[0]
                except:
                    pass
            
            # autor_assistencia
            exp_2 = re.findall(r'assistência .+', exp, flags=re.I)
            if len(exp_2) > 0:
                self.autor_assistencia = re.sub(r' *assistência de ', '',exp_2[0], flags = re.I).replace('.','')
                for p in padrao_inicio_desc:
                    if re.search(p, self.autor_assistencia, flags=re.I) != None:
                        self.autor_assistencia = re.sub(p+'.+','', self.autor_assistencia,flags=re.I)[:-1]  
                        break
                    
            # descricao_assistencia
            exp_2 = re.findall(r'assistência .+', exp, flags=re.I)
            if len(exp_2) > 0:
                for p in padrao_desc_assistencia:
                    try:
                        self.descricao_assistencia = re.findall(p, exp_2,flags=re.I)[0]
                    except:
                        pass
                    
            # origem_finalizacao
            for p in padrao_origem_finalizacao:
                try:
                    self.origem_finalizacao = re.findall(p, exp,flags=re.I)[0]
                except:
                    pass
                
    def gerar_df(self):
        
        if self.bin_finalizacao:
        
            columns = ['jogo_id', 
                       'index',
                       'tempo', 
                       'penalti',
                       'time', 
                       'autor_finalizacao',         
                       'finalizacao',
                       'local_finalizacao',
                       'lado_gol', 
                       'autor_assistencia', 
                       'descricao_assistencia', 
                       'origem_finalizacao']
            
            data = [self.jogo_id, 
                    self.index,
                    self.tempo, 
                    self.bin_penalti,
                    self.time, 
                    self.autor_finalizacao,         
                    self.finalizacao,
                    self.local_finalizacao,
                    self.lado_gol, 
                    self.autor_assistencia, 
                    self.descricao_assistencia, 
                    self.origem_finalizacao]
            
            self.df = pd.DataFrame(data = data).transpose()
            self.df.columns = columns
        
        else:
            return 'Erro! O evento inserido não identifica um gol.'

In [24]:
columns = ['jogo_id', 
           'index',
           'tempo', 
           'penalti',
           'time', 
           'autor_finalizacao',         
           'finalizacao',
           'local_finalizacao',
           'lado_gol', 
           'autor_assistencia', 
           'descricao_assistencia', 
           'origem_finalizacao']

df_finalizacao = pd.DataFrame(columns = columns)

for ex in exemplos_gols:

    jogo_id =  ex['jogo_id']
    req = RequisiçãoFutbeol(jogo_id)
    while req.minuto_a_minuto == None or req.gols == None:
        req.req_minuto_a_minuto()
        req.req_gols()        
    min_a_min = req.minuto_a_minuto
    
    for i in range(len(min_a_min)):        
        fin = Finalizacao(min_a_min[i], jogo_id, i)        
        if fin.bin_finalizacao:
            fin.gerar_df()
            df = fin.df    
            df_finalizacao = pd.concat([df_finalizacao, df])
        
df_finalizacao.index = range(df_finalizacao.shape[0])

In [30]:
jogo_id_0 = None
for l in range(df_finalizacao.shape[0]):
    
    jogo_id = df_finalizacao['jogo_id'][l]
    index = df_finalizacao['index'][l]
    if l > 0:
        jogo_id_0 = df_finalizacao['jogo_id'][l-1]
    
    if jogo_id != jogo_id_0:
        req = RequisiçãoFutbeol(jogo_id)
        req.req_minuto_a_minuto()
   
    min_a_min = req.minuto_a_minuto
    
    print(l+1)
    print(min_a_min[index])
    for c in df_finalizacao.columns:
        if df_finalizacao[c][l] != None and c != 'i' and c != 'j' and c != 'index':
            print(c + ':',df_finalizacao[c][l])
    print('--------------------------------------------')

1
{'tempo': 90, 'descrição': 'Finalização bloqueada, Kevin De Bruyne (Manchester City) finalização com o pé esquerdo de fora da área.'}
jogo_id: 600630
tempo: 90
penalti: False
time: Manchester City
autor_finalizacao: Kevin De Bruyne
finalizacao: pé esquerdo
local_finalizacao: fora da área
lado_gol: Finalização bloqueada
--------------------------------------------
2
{'tempo': 87, 'descrição': 'Finalização defendida em direção ao centro do gol. Phil Foden (Manchester City) finalização com o pé esquerdo do meio da área.'}
jogo_id: 600630
tempo: 87
penalti: False
time: Manchester City
autor_finalizacao: Phil Foden
finalizacao: pé esquerdo
local_finalizacao: meio da área
lado_gol: centro do gol
--------------------------------------------
3
{'tempo': 84, 'descrição': 'Oportunidade perdida Riyad Mahrez (Manchester City), finalização com o pé esquerdo de fora da área.'}
jogo_id: 600630
tempo: 84
penalti: False
time: Manchester City
autor_finalizacao: Riyad Mahrez
finalizacao: pé esquerdo
lo

24
{'tempo': 90, 'descrição': 'Oportunidade perdida Manu Trigueros (Villarreal), finalização com o pé direito de fora da área. Assistência de Moi Gómez.'}
jogo_id: 581882
tempo: 90
penalti: False
time: Villarreal
autor_finalizacao: Manu Trigueros
finalizacao: pé direito
local_finalizacao: fora da área
lado_gol: Oportunidade perdida
autor_assistencia: Moi Gómez
--------------------------------------------
25
{'tempo': 90, 'descrição': 'Finalização bloqueada, Mario Gaspar (Villarreal) finalização com o pé direito do lado direito da área. Assistência de Rubén Peña.'}
jogo_id: 581882
tempo: 90
penalti: False
time: Villarreal
autor_finalizacao: Mario Gaspar
finalizacao: pé direito
local_finalizacao: lado direito da área
lado_gol: Finalização bloqueada
autor_assistencia: Rubén Peña
--------------------------------------------
26
{'tempo': 82, 'descrição': 'Oportunidade perdida Son (Levante), finalização com o pé direito do lado direito da área. Assistência de Roger Martí.'}
jogo_id: 581882
t

46
{'tempo': 87, 'descrição': 'Finalização defendida no lado esquerdo do gol. Aitor Ruibal (Real Betis) finalização com o pé direito do lado esquerdo da área. Assistência de Juan Miranda N.'}
jogo_id: 581881
tempo: 87
penalti: False
time: Real Betis
autor_finalizacao: Aitor Ruibal
finalizacao: pé direito
local_finalizacao: lado esquerdo da área
lado_gol: lado esquerdo do gol
autor_assistencia: Juan Miranda N
--------------------------------------------
47
{'tempo': 84, 'descrição': 'Finalização defendida ao ângulo superior esquerdo. Aitor Ruibal (Real Betis) finalização com o pé direito do meio da área. Assistência de Emerson com um cruzamento.'}
jogo_id: 581881
tempo: 84
penalti: False
time: Real Betis
autor_finalizacao: Aitor Ruibal
finalizacao: pé direito
local_finalizacao: meio da área
lado_gol: ângulo superior esquerdo
autor_assistencia: Emerson
--------------------------------------------
48
{'tempo': 82, 'descrição': 'Finalização bloqueada, Emerson (Real Betis) finalização do me

82
{'tempo': 90, 'descrição': 'Finalização defendida em direção ao centro do gol. John McGinn (Aston Villa) finalização com o pé esquerdo de fora da área. Assistência de Ahmed El Mohamady.'}
jogo_id: 578340
tempo: 90
penalti: False
time: Aston Villa
autor_finalizacao: John McGinn
finalizacao: pé esquerdo
local_finalizacao: fora da área
lado_gol: centro do gol
autor_assistencia: Ahmed El Mohamady
--------------------------------------------
83
{'tempo': 90, 'descrição': 'Finalização defendida em direção ao centro do gol. Wesley (Aston Villa) de cabeça do meio da área. Assistência de Ahmed El Mohamady com um cruzamento.'}
jogo_id: 578340
tempo: 90
penalti: False
time: Aston Villa
autor_finalizacao: Wesley
finalizacao: de cabeça
local_finalizacao: meio da área
lado_gol: centro do gol
autor_assistencia: Ahmed El Mohamady
--------------------------------------------
84
{'tempo': 89, 'descrição': 'Oportunidade perdida Conor Gallagher (West Bromwich Albion), finalização com o pé esquerdo de f

113
{'tempo': 90, 'descrição': 'Finalização bloqueada, Bruno Fernandes (Manchester United) finalização com o pé direito do lado direito da pequena área.'}
jogo_id: 578342
tempo: 90
penalti: False
time: Manchester United
autor_finalizacao: Bruno Fernandes
finalizacao: pé direito
local_finalizacao: lado direito da pequena área
lado_gol: Finalização bloqueada
--------------------------------------------
114
{'tempo': 90, 'descrição': 'Finalização defendida em direção ao centro do gol. Mason Greenwood (Manchester United) finalização com o pé esquerdo do lado direito da pequena área.'}
jogo_id: 578342
tempo: 90
penalti: False
time: Manchester United
autor_finalizacao: Mason Greenwood
finalizacao: pé esquerdo
local_finalizacao: lado direito da pequena área
lado_gol: centro do gol
--------------------------------------------
115
{'tempo': 90, 'descrição': 'Finalização defendida em direção ao centro do gol. Scott McTominay (Manchester United) finalização com o pé direito do meio da área. Assis

...

188
{'tempo': 85, 'descrição': 'Finalização bloqueada, Mohamed Fares (Lazio) finalização com o pé direito do lado direito da área. Assistência de Ciro Immobile.'}
jogo_id: 582824
tempo: 85
penalti: False
time: Lazio
autor_finalizacao: Mohamed Fares
finalizacao: pé direito
local_finalizacao: lado direito da área
lado_gol: Finalização bloqueada
autor_assistencia: Ciro Immobile
--------------------------------------------
189
{'tempo': 77, 'descrição': 'Finalização defendida em direção ao centro do gol. Ciro Immobile (Lazio) finalização com o pé direito do meio da área. Assistência de Manuel Lazzari.'}
jogo_id: 582824
tempo: 77
penalti: False
time: Lazio
autor_finalizacao: Ciro Immobile
finalizacao: pé direito
local_finalizacao: meio da área
lado_gol: centro do gol
autor_assistencia: Manuel Lazzari
--------------------------------------------
190
{'tempo': 76, 'descrição': 'Finalização defendida em direção ao centro do gol. Francesco Cassata (Genoa) finalização com o pé direito de fora da

211
{'tempo': 90, 'descrição': 'Finalização defendida junto ao lado inferior direito do gol. Kasper Dolberg (Nice) finalização com o pé direito do lado direito da área. Assistência de Alexis Claude Maurice com um passe em profundidade.'}
jogo_id: 572939
tempo: 90
penalti: False
time: Nice
autor_finalizacao: Kasper Dolberg
finalizacao: pé direito
local_finalizacao: lado direito da área
lado_gol: lado inferior direito do gol
autor_assistencia: Alexis Claude Maurice
--------------------------------------------
212
{'tempo': 90, 'descrição': 'Finalização defendida no lado esquerdo do gol. Dan Ndoye (Nice) finalização com o pé direito do meio da área. Assistência de Flavius Daniliuc.'}
jogo_id: 572939
tempo: 90
penalti: False
time: Nice
autor_finalizacao: Dan Ndoye
finalizacao: pé direito
local_finalizacao: meio da área
lado_gol: lado esquerdo do gol
autor_assistencia: Flavius Daniliuc
--------------------------------------------
213
{'tempo': 87, 'descrição': 'Oportunidade perdida Pierre L

239
{'tempo': 90, 'descrição': 'Finalização defendida junto ao lado inferior direito do gol. Hatem Ben Arfa (Bordeaux) finalização com o pé esquerdo de fora da área. Assistência de Paul Baysse.'}
jogo_id: 572934
tempo: 90
penalti: False
time: Bordeaux
autor_finalizacao: Hatem Ben Arfa
finalizacao: pé esquerdo
local_finalizacao: fora da área
lado_gol: lado inferior direito do gol
autor_assistencia: Paul Baysse
--------------------------------------------
240
{'tempo': 90, 'descrição': 'Finalização bloqueada, Amadou Traoré (Bordeaux) finalização com o pé direito do lado direito da área.'}
jogo_id: 572934
tempo: 90
penalti: False
time: Bordeaux
autor_finalizacao: Amadou Traoré
finalizacao: pé direito
local_finalizacao: lado direito da área
lado_gol: Finalização bloqueada
--------------------------------------------
241
{'tempo': 90, 'descrição': 'Oportunidade perdida Hatem Ben Arfa (Bordeaux), finalização com o pé esquerdo de fora da área de falta.'}
jogo_id: 572934
tempo: 90
penalti: Fal

260
{'tempo': 90, 'descrição': 'Finalização bloqueada, Chimuanya Ugochukwu (Rennes) finalização com o pé direito do meio da área. Assistência de Flavien Tait.'}
jogo_id: 572936
tempo: 90
penalti: False
time: Rennes
autor_finalizacao: Chimuanya Ugochukwu
finalizacao: pé direito
local_finalizacao: meio da área
lado_gol: Finalização bloqueada
autor_assistencia: Flavien Tait
--------------------------------------------
261
{'tempo': 87, 'descrição': 'Finalização bloqueada, Steven Nzonzi (Rennes) finalização com o pé direito de fora da área. Assistência de Jéremy Doku.'}
jogo_id: 572936
tempo: 87
penalti: False
time: Rennes
autor_finalizacao: Steven Nzonzi
finalizacao: pé direito
local_finalizacao: fora da área
lado_gol: Finalização bloqueada
autor_assistencia: Jéremy Doku
--------------------------------------------
262
{'tempo': 78, 'descrição': 'Finalização defendida em direção ao centro do gol. Jéremy Doku (Rennes) finalização com o pé direito do lado esquerdo da área. Assistência de St

283
{'tempo': 89, 'descrição': 'Finalização bloqueada, Marcelo (Lyon) de cabeça do meio da área. Assistência de Memphis Depay com um cruzamento.'}
jogo_id: 572937
tempo: 89
penalti: False
time: Lyon
autor_finalizacao: Marcelo
finalizacao: de cabeça
local_finalizacao: meio da área
lado_gol: Finalização bloqueada
autor_assistencia: Memphis Depay
--------------------------------------------
284
{'tempo': 88, 'descrição': 'Finalização bloqueada, Maxwel Cornet (Lyon) de cabeça do meio da área. Assistência de Melvin Bard com um cruzamento.'}
jogo_id: 572937
tempo: 88
penalti: False
time: Lyon
autor_finalizacao: Maxwel Cornet
finalizacao: de cabeça
local_finalizacao: meio da área
lado_gol: Finalização bloqueada
autor_assistencia: Melvin Bard
--------------------------------------------
285
{'tempo': 77, 'descrição': 'Oportunidade perdida Jonathan Bamba (Lille), finalização com o pé direito do lado esquerdo da área.'}
jogo_id: 572937
tempo: 77
penalti: False
time: Lille
autor_finalizacao: Jona

In [36]:
jogo_id = '572939'
req = RequisiçãoFutbeol(jogo_id)

In [37]:
req.req_jogadores()

In [39]:
req.jogadores[0]

[{'nome': 'Walter Benítez',
  'camisa': 40,
  'inicio_jogando': 0,
  'final_jogando': 90},
 {'nome': 'William Saliba',
  'camisa': 18,
  'inicio_jogando': 0,
  'final_jogando': 90},
 {'nome': 'Jean-Clair Todibo',
  'camisa': 25,
  'inicio_jogando': 0,
  'final_jogando': 90},
 {'nome': 'Morgan Schneiderlin',
  'camisa': 6,
  'inicio_jogando': 0,
  'final_jogando': 90},
 {'nome': 'Hassane Kamara',
  'camisa': 13,
  'inicio_jogando': 0,
  'final_jogando': 90},
 {'nome': 'Jordan Lotomba',
  'camisa': 23,
  'inicio_jogando': 0,
  'final_jogando': 80},
 {'nome': 'Dan Ndoye',
  'camisa': 14,
  'inicio_jogando': 80,
  'final_jogando': 90},
 {'nome': 'Hichem Boudaoui',
  'camisa': 28,
  'inicio_jogando': 0,
  'final_jogando': 64},
 {'nome': 'Pierre Lees-Melou',
  'camisa': 8,
  'inicio_jogando': 64,
  'final_jogando': 90},
 {'nome': 'Alexis Claude Maurice',
  'camisa': 10,
  'inicio_jogando': 0,
  'final_jogando': 90},
 {'nome': 'Rony Lopes',
  'camisa': 22,
  'inicio_jogando': 0,
  'final_joga